<a href="https://colab.research.google.com/github/detauportuno/Projeto_Pinturas_PAE/blob/main/Aula3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução à inteligência artificial

## Aula 3 - Introdução ao PyTorch

Neste caderno, você será apresentado ao [PyTorch](http://pytorch.org/), uma estrutura para construir e treinar redes neurais. O PyTorch se comporta de várias maneiras como as matrizes que você ama do Numpy. Afinal, essas matrizes Numpy são apenas tensores. O PyTorch pega esses tensores e facilita a sua transferência para GPUs para o processamento mais rápido necessário ao treinar redes neurais. Ele também fornece um módulo que calcula automaticamente gradientes (para retropropagação!) E outro módulo especificamente para a construção de redes neurais. No conjunto, o PyTorch acaba sendo mais coerente com o Python e a pilha Numpy/Scipy em comparação com o TensorFlow e outras estruturas.

### Tensores

Acontece que os cálculos de redes neurais são apenas um monte de operações de álgebra linear em *tensores*, uma generalização de matrizes. Um vetor é um tensor unidimensional, uma matriz é um tensor bidimensional, uma matriz com três índices é um tensor tridimensional (imagens em cores RGB, por exemplo). A estrutura de dados fundamental para redes neurais são os tensores e o PyTorch (assim como praticamente qualquer outra estrutura de aprendizado profundo) é construído em torno dos tensores.

Com o básico abordado, é hora de explorar como podemos usar o PyTorch para construir uma rede neural simples.

In [ ]:
# Vamos importar o PyTorch
import torch

In [ ]:
def activation(x):
    """ Função de ativação sigmóide

        Argumentos
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

In [ ]:
### Gere alguns dados
torch.manual_seed(7) # Defina a semente aleatória para que as coisas sejam previsíveis

# Recursos são 5 variáveis normais aleatórias
features = torch.randn((1, 5))
# Pesos verdadeiros para nossos dados, variáveis normais aleatórias novamente
weights = torch.randn_like(features)
# e um termo verdadeiro viés
bias = torch.randn((1, 1))

Acima, geramos dados que podemos usar para obter a saída de nossa rede simples. Por enquanto, tudo é aleatório. No futuro, começaremos a usar dados normais. Passando por cada linha relevante:

`features = torch.randn((1, 5))` cria um tensor com a forma `(1, 5)`, uma linha e cinco colunas, que contém valores distribuídos aleatoriamente de acordo com a distribuição normal, com média de zero e padrão desvio de um.

`weights = torch.randn_like(features)` cria outro tensor com a mesma forma que `features`, novamente contendo valores de uma distribuição normal.

Finalmente, `bias = torch.randn((1, 1))` cria um valor único a partir de uma distribuição normal.

Os tensores PyTorch podem ser adicionados, multiplicados, subtraídos, etc., assim como os arrays Numpy. Em geral, você usará os tensores PyTorch da mesma maneira que usaria as matrizes Numpy. Eles trazem alguns benefícios interessantes, como a aceleração da GPU, que veremos mais adiante. Por enquanto, use os dados gerados para calcular a saída dessa rede simples de camada única.
> **Exemplo**: Calcule a saída da rede com recursos de entrada `features`, pesos `weights` e viés `bias`. Semelhante ao Numpy, o PyTorch possui a função [`torch.sum()`](https://pytorch.org/docs/stable/torch.html#torch.sum), além do método `.sum()` em tensores, para somar os resultados. Use a função 'activation' definida acima como a função de ativação.

In [ ]:
output = activation(torch.sum(weights*features)+bias)
output

Você pode fazer a multiplicação e somar na mesma operação usando uma multiplicação de matrizes. Em geral, convém usar multiplicações de matriz, pois elas são mais eficientes e aceleradas usando bibliotecas modernas e computação de alto desempenho em GPUs.

Aqui, queremos fazer uma multiplicação matricial dos recursos e pesos. Para isso, podemos usar [`torch.mm()`](https://pytorch.org/docs/stable/torch.html#torch.mm) ou [`torch.matmul()`](https://pytorch.org/docs/stable/torch.html#torch.matmul), que é um pouco mais complicado e suporta transmissão. Se tentarmos fazê-lo com os 'features' e 'weights' como eles são, obteremos um erro:

```python
>> torch.mm(features, weights)

---------------------------------------------------------------------------
RuntimeError                              Traceback (most recent call last)
<ipython-input-13-15d592eb5279> in <module>()
----> 1 torch.mm(features, weights)

RuntimeError: size mismatch, m1: [1 x 5], m2: [1 x 5] at /Users/soumith/minicondabuild3/conda-bld/pytorch_1524590658547/work/aten/src/TH/generic/THTensorMath.c:2033
```

Ao criar redes neurais em qualquer estrutura, você verá isso com frequência. Realmente frequentemente. O que está acontecendo aqui é que nossos tensores não têm as formas corretas para realizar uma multiplicação de matrizes. Lembre-se de que para multiplicações de matrizes, o número de colunas no primeiro tensor deve ser igual ao número de linhas na segunda coluna. Ambos os `features` e `weights` têm a mesma forma, `(1, 5)`. Isso significa que precisamos alterar a forma dos pesos para que a multiplicação da matriz funcione.

**Nota:** Para ver a forma de um tensor chamado `tensor`, use `tensor.shape`. Se você estiver construindo redes neurais, estará usando esse método frequentemente.

Existem algumas opções aqui: [`weights.reshape()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.reshape), [`weights.resize_()`]( https://pytorch.org/docs/stable/tensors.html#torch.Tensor.resize_) e [`weights.view()`](https://pytorch.org/docs/stable/tensors.html#torch.Tensor.view).

* `weights.reshape(a, b)` retornará um novo tensor com os mesmos dados que `weights` com tamanho `(a, b)` às vezes e às vezes um clone, pois ele copia os dados para outra parte do memória.
* `weights.resize_(a, b)` retorna o mesmo tensor com uma forma diferente. No entanto, se a nova forma resultar em menos elementos que o tensor original, alguns elementos serão removidos do tensor (mas não da memória). Se a nova forma resultar em mais elementos que o tensor original, novos elementos serão não inicializados na memória. Aqui, devo observar que o sublinhado no final do método indica que esse método é realizado ** no local **. Aqui está um ótimo tópico do fórum para [leia mais sobre operações locais](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) no PyTorch.
* `weights.view(a, b)` retornará um novo tensor com os mesmos dados que `weights` com tamanho` (a, b) `.

Portanto, agora podemos remodelar 'weights' para ter cinco linhas e uma coluna com algo como 'weights.view(5, 1)'.

> **Exemplo**: Calcule a saída da nossa pequena rede usando multiplicação de matrizes.

In [ ]:
output = activation(torch.mm(features,weights.view(5, 1))+bias)
output

### Junte tudo
É assim que você pode calcular a saída para um único neurônio. O poder real desse algoritmo acontece quando você começa a empilhar essas unidades individuais em camadas e pilhas de camadas, em uma rede de neurônios. A saída de uma camada de neurônios se torna a entrada para a próxima camada. Com várias unidades de entrada e unidades de saída, agora precisamos expressar os pesos como uma matriz.

A primeira camada mostrada na parte inferior aqui são as entradas, compreensivelmente chamadas de **camada de entrada**. A camada do meio é chamada de **camada oculta** e a camada final (à direita) é a **camada de saída**. Podemos expressar essa rede matematicamente com matrizes novamente e usar a multiplicação de matrizes para obter combinações lineares para cada unidade em uma operação. Por exemplo, a camada oculta ($ h_1 $ e $ h_2 $ aqui) pode ser

$$
\vec{h} = [h_1 \, h_2] =
\begin{bmatrix}
x_1 \, x_2 \cdots \, x_n
\end{bmatrix}
\cdot
\begin{bmatrix}
           w_{11} & w_{12} \\
           w_{21} &w_{22} \\
           \vdots &\vdots \\
           w_{n1} &w_{n2}
\end{bmatrix}
$$

A saída para esta pequena rede é encontrada tratando a camada oculta como entradas para a unidade de saída. A saída da rede é expressa simplesmente

$$
y =  f_2 \! \left(\, f_1 \! \left(\vec{x} \, \mathbf{W_1}\right) \mathbf{W_2} \right)
$$


In [ ]:
torch.manual_seed(7)

# Os 'features' são três variáveis normais aleatórias
features = torch.randn((1, 3))

# Defina o tamanho de cada camada em nossa rede
n_input = features.shape[1]     # Número de unidades de entrada, deve corresponder ao número de 'features' de entrada
n_hidden = 2                    # Número de unidades ocultas
n_output = 1                    # Número de unidades de saída

# 'Weights' para entradas na camada oculta
W1 = torch.randn(n_input, n_hidden)
# 'Weights' da camada oculta para a camada de saída
W2 = torch.randn(n_hidden, n_output)

# e 'bias' para camadas ocultas e de saída
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

> **Exercício:** Calcule a saída desta rede de múltiplas camadas usando os 'weights' `W1` e `W2`, e os 'bias', `B1` e `B2`.

In [ ]:
h = None
print(h)
output = None
print(output)

Se você fez isso corretamente, deverá ver a saída da rede igual a `tensor([[0,3171]]) '.

O número de unidades ocultas é um parâmetro da rede, geralmente chamado de **hiperparâmetro** para diferenciá-lo dos parâmetros de pesos e desvios. Como você verá mais adiante, quando discutirmos o treinamento de uma rede neural, quanto mais unidades ocultas uma rede tiver e mais camadas, maior será a capacidade de aprender com os dados e fazer previsões precisas.


## Redes neurais com PyTorch

As redes de aprendizado profundo tendem a ser massivas com dezenas ou centenas de camadas, é daí que o termo "profundo" vem. Você pode construir uma dessas redes profundas usando apenas matrizes de peso, como fizemos no notebook anterior, mas, em geral, é muito complicado e difícil de implementar. O PyTorch possui um ótimo módulo `nn`, que fornece uma boa maneira de construir eficientemente grandes redes neurais.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
from torch import nn

import matplotlib.pyplot as plt

Agora vamos construir uma rede maior que possa resolver um problema (anteriormente) difícil, identificando texto em uma imagem. Aqui, usaremos o conjunto de dados MNIST que consiste em dígitos manuscritos em escala de cinza. Cada imagem tem 28x28 pixels. Você pode ver uma amostra abaixo

In [ ]:
from torchvision import datasets, transforms

# Defina uma transformação para normalizar os dados
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Faça o download e carregue os dados de treinamento
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

Você notará que foi criado o `trainloader` com um tamanho de lote de 64 e `shuffle = True`. O tamanho do lote é o número de imagens que obtemos em uma iteração do carregador de dados e passamos por nossa rede, geralmente chamada de *batch*. E `shuffle = True` diz para ele embaralhar o conjunto de dados toda vez que começamos a percorrer o carregador de dados novamente. Mas aqui estou apenas pegando o primeiro lote para que possamos verificar os dados. Podemos ver abaixo que `images` é apenas um tensor com tamanho `(64, 1, 28, 28) `. Portanto, 64 imagens por lote, 1 canal de cor e imagens 28x28.

In [ ]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images)) # Tipo da informação
print(images.shape) # Quantidade de imagens e tamanho de cada imagem
print(labels.shape) # Quantidade de 'labels'

In [ ]:
# Mostrar imagem aleatória
plt.imshow(images[2].numpy().squeeze(), cmap='Greys_r');

Primeiro, vamos tentar construir uma rede simples para esse conjunto de dados usando matrizes de peso e multiplicações de matrizes. Em seguida, veremos como fazer isso usando o módulo `nn` do PyTorch, que fornece um método muito mais conveniente e poderoso para definir arquiteturas de rede.

As redes que você viu até agora são chamadas de redes *totalmente conectadas* ou *densas*. Cada unidade em uma camada é conectada a cada unidade na próxima camada. Em redes totalmente conectadas, a entrada para cada camada deve ser um vetor unidimensional (que pode ser empilhado em um tensor 2D como um lote de vários exemplos). No entanto, nossas imagens são tensores 2D de 28x28, portanto, precisamos convertê-los em vetores 1D. Pensando nos tamanhos, precisamos converter o lote de imagens com a forma `(64, 1, 28, 28)` para uma forma de `(64, 784)`, 784 é 28 vezes 28. Isso é normalmente chamado de *achatamento*, achatamos as imagens 2D em vetores 1D.

Anteriormente, você construiu uma rede com uma unidade de saída. Aqui precisamos de 10 unidades de saída, uma para cada dígito. Queremos que nossa rede preveja o dígito mostrado em uma imagem; portanto, o que faremos é calcular as probabilidades de que a imagem seja de qualquer dígito ou classe. Isso acaba sendo uma distribuição de probabilidade discreta nas classes (dígitos) que nos diz a classe mais provável para a imagem. Isso significa que precisamos de 10 unidades de saída para as 10 classes (dígitos). Veremos como converter a saída da rede em uma distribuição de probabilidade a seguir.

> **Exercício:** Achatar o lote de imagens `images`. Em seguida, construa uma rede de várias camadas com 784 unidades de entrada, 256 unidades ocultas e 10 unidades de saída usando tensores aleatórios para os pesos e desvios. Por enquanto, use uma ativação sigmóide para a camada oculta. Deixe a camada de saída sem uma ativação; adicionaremos uma que nos fornecerá uma distribuição de probabilidade a seguir.

In [ ]:
def activation(x):
    """ Função de ativação sigmóide

        Argumentos
        ---------
        x: torch.Tensor
    """
    return 1/(1+torch.exp(-x))

imgVectorized = images.view(64,-1) # Faça o "achatamento" do lote de imagem


n_input = imgVectorized.shape[1]     # Número de unidades de entrada, deve corresponder ao número de 'features' de entrada
n_hidden = 256                    # Número de unidades ocultas
n_output = 10                    # Número de unidades de saída

# Weights
W1 = torch.randn(n_input, n_hidden)
W2 = torch.randn(n_hidden, n_output)

# Bias
B1 = torch.randn((1, n_hidden))
B2 = torch.randn((1, n_output))

h = activation(torch.mm(imgVectorized,W1)+B1)
out = activation(torch.mm(h,W2)+B2)

out.size()
# O resultado deve ser 'torch.Size([64, 10])'

Agora temos 10 saídas para nossa rede. Queremos passar uma imagem para a nossa rede e obter uma distribuição de probabilidade nas classes que nos diz as classes prováveis às quais a imagem pertence. Algo que se parece com isso:

Aqui vemos que a probabilidade para cada classe é aproximadamente a mesma. Isso representa uma rede não treinada, mas ainda não viu nenhum dado, apenas retorna uma distribuição uniforme com probabilidades iguais para cada classe.

Para calcular essa distribuição de probabilidade, geralmente usamos a função [**softmax**](https://en.wikipedia.org/wiki/Softmax_function). Matematicamente isso parece:


$$
\Large \sigma(x_i) = \cfrac{e^{x_i}}{\sum_k^K{e^{x_k}}}
$$


O que isso faz é espremer cada entrada $ x_i $ entre 0 e 1 e normaliza os valores para fornecer uma distribuição de probabilidade adequada, na qual as probabilidades atingem um.

> **Exercício:** Implemente uma função `softmax` que execute o cálculo do softmax e retorne as distribuições de probabilidade para cada exemplo no lote. Observe que você precisará prestar atenção às formas ao fazer isso. Se você tiver um tensor `a` com forma `(64, 10)` e um tensor `b` com forma `(64,)`, fazer `a / b` gerará um erro porque o PyTorch tentará fazer o divisão nas colunas (chamada de transmissão), mas você encontrará uma incompatibilidade de tamanho. A maneira de pensar sobre isso é para cada um dos 64 exemplos, você só deseja dividir por um valor, a soma no denominador. Então você precisa que `b` tenha a forma de` (64, 1) `. Dessa forma, o PyTorch dividirá os 10 valores em cada linha de `a` pelo valor único em cada linha de` b`. Preste atenção em como você recebe a soma também. Você precisará definir a palavra-chave `dim` em` torch.sum`. Definir `dim=0` leva a soma entre as linhas enquanto  `dim=1` leva a soma entre as colunas.

In [ ]:
def softmax(x):
    ## TODO: Implementar softmax aqui
    exp_tensor = torch.exp(x)
    sum_exp_tensor = torch.sum(torch.exp(x), dim=) # As somas devem ser entre colunas
    return exp_tensor/sum_exp_tensor.view(-1, 1)

probabilities = softmax(out)

# Tem a forma correta? Deve ser (64, 10)
print(probabilities.shape)
# A soma equivale a 1?
print(probabilities.sum(dim=1))

SyntaxError: ignored

Até agora, apenas observamos a ativação do softmax, mas em geral qualquer função pode ser usada como uma função de ativação. O único requisito é que, para que uma rede se aproxime de uma função não linear, as funções de ativação devem ser não lineares. Aqui estão mais alguns exemplos de funções de ativação comuns: Tanh (tangente hiperbólica) e ReLU (unidade linear retificada).

![activations](https://www.kdnuggets.com/wp-content/uploads/activation.png)

Na prática, a função ReLU é usada quase exclusivamente como a função de ativação para camadas ocultas.

### Criando a rede neural

![neural](https://raw.githubusercontent.com/dmlc/web-data/master/mxnet/image/mlp_mnist.png)

> **Exercício:** Crie uma rede com 784 unidades de entrada, uma camada oculta com 128 unidades e uma ativação ReLU, depois uma camada oculta com 64 unidades e uma ativação ReLU e, finalmente, uma camada de saída com uma ativação softmax como mostrado acima . Você pode usar uma ativação ReLU com o módulo `nn.ReLU` ou a função` F.relu`.

In [ ]:
def view_classify(img, ps, version="MNIST"):
    ps = ps.data.numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(img.resize_(1, 28, 28).numpy().squeeze())
    ax1.axis('off')
    ax2.barh(np.arange(10), ps)
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    if version == "MNIST":
        ax2.set_yticklabels(np.arange(10))
    ax2.set_title('Probabilidade de classe')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()

In [ ]:
# Hiperparâmetros para nossa rede
input_size = None
hidden_sizes = [None, None] #vetor com duas posições
output_size = None

# Construa uma rede de feed-forward
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size))
print(model)

# Passagem direta pela rede e mostra imagem
images, labels = next(iter(trainloader))
images.resize_(images.shape[0], 1, 784)
m = nn.Softmax(dim=1)
ps = m(model.forward(images[0,:]))
view_classify(images[0].view(1, 28, 28), ps)

In [ ]:
#@title Treine a rede
from torch import optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
epochs = 5
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print(f"Perda de treinamento: {running_loss/len(trainloader)}")

In [ ]:
#@title Execute para ver o resultado
images, labels = next(iter(trainloader))
img = images[1].view(1, 784)
with torch.no_grad():
    logps = model(img)
m = nn.Softmax(dim=1)
ps = m(logps)
view_classify(img.view(1, 28, 28), ps)